During development of [NI Water](https://github.com/andrewbolster/bolster/pull/1009), the dataset changed underfoot and became disconnected from the [OpenDataNI dataset](https://admin.opendatani.gov.uk/dataset/ni-water-customer-tap-authorised-supply-point-results) that drove the mapping between Zones and Postcodes

Presumably, it's a load of postcodes (all BT34) that didn't participate in the 2023, but did in 2022, which doesn't make any sense either because the api call ("https://www.niwater.com/water-quality-lookup.ashx?z={zone_code}") is just returning blank. (Zone ZS0904)

Anyway, this is here just to remind me when this all goes to hell next time. 

In [ ]:
import sys
import os
from pathlib import Path

# Get the absolute path to the src directory
# Adjust this path as needed based on the location of your notebook
src_path = Path("../src").resolve()  # If scripts is at the same level as src
sys.path.insert(0, str(src_path))

import bolster

from bolster.data_sources.ni_water import *

supply_zones = set(get_postcode_to_water_supply_zone().values())

df = pd.DataFrame([get_water_quality_by_zone(zone_code) for zone_code in supply_zones if zone_code != "No Zone Identified"])
df = df.astype({"NI Hardness Classification": T_HARDNESS})
df

In [ ]:
df

In [ ]:
df = get_water_quality()

In [ ]:
df

In [ ]:
mapping = get_postcode_to_water_supply_zone()

In [ ]:
{ code for code, zone in mapping.items() if zone == 'ZS0904'}

In [ ]:
data = get_water_quality_by_zone('ZS0101')

In [ ]:
data.index

In [ ]:
df['NI Hardness Classification'].value_counts().keys()